In [ ]:
PATH_IMAGES = "./data/imgs/"

#EPOCHS = 80
#BATCH_SIZE = 5

import os

#os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [ ]:
#import tensorflow as tf
#import tensorflow_lattice as tfl
#print(tf.__version__)
#print(tf.keras.backend.image_data_format())
import scipy.io 

from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from tqdm import tqdm
tqdm.pandas()
import numpy as np

from PIL import Image

import random
import sys
import math

import cv2
import cvlib
import dlib

In [ ]:
if os.path.isfile("./data/metadata.pkl"):
    df = pd.read_pickle("./data/metadata.pkl")
else:
    data = scipy.io.loadmat('./data/metadata.mat')
    print(data.keys())
    mappingSplits = {0: data["splits"][0][0][0], 1: data["splits"][0][1][0], 2: data["splits"][0][2][0], 3:data["splits"][0][3][0]}
    print(mappingSplits)

    df = pd.DataFrame([data["recording"][0], data["split"][0], data["gaze_dir"], data["frame"][0], data["person_identity"][0]]).T
    df.columns =  ["Recording", "SplitId", "GazeDir", "Frame", "PersonIdentity"]
    df["Path"] = df.apply(lambda e: "%srec_%03d/head/%06d/%06d.jpg" % (PATH_IMAGES, e.Recording, e.PersonIdentity, e.Frame), axis=1)
    df["Split"] = df.SplitId.map(mappingSplits)
    df["GazeDirNorm"] = df.GazeDir.apply(lambda x: x) # this seams to have no effect normalize()
    df["Gaze"] = df.GazeDirNorm.apply(lambda x: np.array([math.atan2(x[0],-x[2]), math.asin(x[1])]))
    df["GazeAngle"] = df.Gaze.apply(lambda x: np.rad2deg(x))
    df[["Split", "Gaze", "Path"]].to_pickle("./data/metadata_small.pkl")
    df.to_pickle("./data/metadata.pkl")
    df.head()

In [ ]:
yaw = df.GazeAngle.apply(lambda x: x[0]).to_list()
pitch = df.GazeAngle.apply(lambda x: x[1]).to_list()
plt.hist(yaw, alpha = .5, label="Yaw")
plt.hist(pitch, alpha = .5, label="Pitch")
plt.legend()
plt.show()
plt.hist2d(yaw, pitch, bins=[np.arange(-180,180,2),np.arange(-90,90,2)], cmap="hot")
plt.colorbar()
plt.show()

In [ ]:
%%time
def countMissingFrames(df, f):
    dfX = df[(df.Frame >= f-3) & (df.Frame <= f+3)]
    return 7 - len(dfX)
lst = []
for i, r in enumerate(sorted(df.Recording.unique())):
    if (i % 5 == 0):
        print("%s: itr %i, Running recoding %i" % (datetime.now(), i, r))
    df1 = df[(df.Recording == r)]
    for p in sorted(df1.PersonIdentity.unique()):
        df2 = df1[(df1.PersonIdentity == p)]
        ret = df2.Frame.apply(lambda x: countMissingFrames(df2, x))
        lst.append(ret)
        
df["MissingCount"] = pd.concat(lst)

In [ ]:
for i in range(0, 7):
    length = len(df[df.MissingCount == i])
    print("%i missing images for %i images %.2f%%" % (i, length, length/ len(df)* 100))
df.MissingCount.hist(bins=6)

In [ ]:
df.groupby(["Split", "MissingCount"]).count()

It is not quite clear why 8702 good samples are not used. 

In [ ]:
# Random test image
if True:
    fig, ax = plt.subplots(3,5, figsize=(18,8))
    for i in range (3):
        for j in range (5):
            path = df[(df.PersonIdentity == 0) & (df.Recording == 1)].sample().iloc[0].Path
            img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
            ax[i][j].imshow(img)
            #faces, confidences = cvlib.detect_face(img)
            #for f in faces:
            #    ax[i][j].add_patch(patches.Rectangle((f[0], f[1]), f[2] - f[0], f[3] - f[1], linewidth=2, edgecolor='r', facecolor='none'))


In [ ]:
#! wget http://arunponnusamy.com/files/mmod_human_face_detector.dat
cnn_face_detector = dlib.cnn_face_detection_model_v1("./mmod_human_face_detector.dat")

In [ ]:
%%time
def getFace(path):
    img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    SIZE_ORG = img.shape
    #faces, confidences = cvlib.detect_face(img, enable_gpu=False)
    #cvlib.detect_face(img)
    #cvlib.detect_face(img, enable_gpu=False) # not precice
    
    #import face_recognition
    #face_recognition.face_locations(img, number_of_times_to_upsample=2, model="cnn") # super slow
    #face_recognition.batch_face_locations(frames, number_of_times_to_upsample=0) # Faster but combersome to use
    
    #https://www.arunponnusamy.com/cnn-face-detector-dlib.html
    
    SIZE = 500
    img2 = cv2.resize(img, (SIZE,SIZE))
    x = cnn_face_detector(img2)
    
    faces = [[a.rect.left()/SIZE*SIZE_ORG[0], a.rect.top()/SIZE*SIZE_ORG[0], a.rect.right()/SIZE*SIZE_ORG[1], a.rect.bottom()/SIZE*SIZE_ORG[1]] for a in x]
    conf = [a.confidence for a in x]
    return faces, conf, SIZE_ORG

df["FaceData"] = df.Path.progress_apply(lambda path: getFace(path))

In [ ]:
df["Faces"] = df.FaceData.apply(lambda x: x[0])
df["FacesConfidence"] = df.FaceData.apply(lambda x: x[1])
df["Size"] = df.FaceData.apply(lambda x: x[2])
df["FaceCount"] = df.Faces.apply(lambda x: len(x))

In [ ]:
df["FaceCount"].hist()

In [ ]:
print("Not recognized: %.2f" %(len(df[df["FaceCount"] == 0])/ len(df) * 100))

In [ ]:
# We know for the given dataset the face has to be in the center of the frame
# Thus, we can use the face which is closest to the center in case the face detaction found multiple faces
# Demo for the next Setp:

e = df[df.FaceCount==2].sample().iloc[0]
path = e.Path
img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
plt.imshow(img)
s = np.array(e.Size[:2])/2
plt.scatter([s[0]],[s[1]], c="r", marker="*",label="Centre")
lst = []
for i, f in enumerate(e.Faces):
    f2 = np.array(f).reshape(2,2)
    plt.scatter(f2[:,0], f2[:,1], label="Face %i" % i)
    
    x, y = f[0], f[1]
    w =  f[2] - f[0]
    h = f[3] - f[1]
    x1,y1 = x + w/2, y + h/2
    distance = math.sqrt( ((x1-s[0])**2)+((y1-s[1])**2) )
    print(i, distance)
    plt.scatter([x1],[y1], marker="x", label="Centre of %i" % i)
    lst.append(distance)
plt.legend()
idx = np.argmin(lst)
print(idx, e.Faces[idx])

In [ ]:
def getOneFace(faces, size, confidence):
    if len(faces) == 0:
        return None, None
    elif len(faces) == 1:
        return faces[0], confidence[0]
    else:
        lst = []
        sizes = []
        s = np.array(size[:2])/2
        for f in faces:
            x, y = f[0], f[1]
            w =  f[2] - f[0]
            h = f[3] - f[1]
            sizes.append((w+h)/2)
            x1,y1 = x + w/2, y + h/2
            distance = math.sqrt( ((x1-s[0])**2)+((y1-s[1])**2) )
            lst.append(distance)
        
        # No we use the bigger one of the ones close to the center
        # This can and has to be done also with later validation datasets
        # Thus, this makes the choise consistent and unique
        lst = np.array(lst)
        norm = lst / s[0]
        idxs = np.argwhere(norm<.2)
        idxs = np.ravel(idxs) # only indcies with a low distance to the centre - Dataset assumtiopn
       
        if len(idxs) < 2:
            idx = np.argmin(lst)
            return faces[idx], confidence[idx]
        else:
            sizes = np.array(sizes)[idxs]
            idx = np.argmax(sizes) # Pick the biggest one. Do this for your database
            return faces[idxs[idx]], confidence[idxs[idx]]
        
x = df.apply(lambda e: getOneFace(e.Faces, e.Size, e.FacesConfidence), axis = 1)   
df["Face"] = x.apply(lambda x: x[0])
df["FaceConfidence"] = x.apply(lambda x: x[1])

In [ ]:
# Random test image
if True:
    fig, ax = plt.subplots(5,5, figsize=(18,18))
    for i in range (5):
        for j in range (5):
            e = df[df.FaceCount==2].sample().iloc[0]
            path = e.Path
            img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
            ax[i][j].imshow(img)
            
            if e.FaceCount > 1:
                for k, f in enumerate(e.Faces):
                    #print(f[2] - f[0], f[3] - f[1])
                    ax[i][j].add_patch(patches.Rectangle((f[0], f[1]), f[2] - f[0], f[3] - f[1], linewidth=2, edgecolor='y', facecolor='none'))
                    ax[i][j].text(f[0], f[1], str(np.round(e.FacesConfidence[k], 2)), color="y", size=14)
            
            f = e.Face
            ax[i][j].add_patch(patches.Rectangle((f[0], f[1]), f[2] - f[0], f[3] - f[1], linewidth=2, edgecolor='r', facecolor='none'))
            w =  f[2] - f[0]
            h = f[3] - f[1]

            ax[i][j].add_patch(patches.Rectangle((f[0]-w*.4, f[1]-h*.4),w*1.8, h*1.8, linewidth=2, edgecolor='w', facecolor='none'))
            ax[i][j].text(f[0], f[1], str(np.round(e.FaceConfidence, 2)), color="r", size=14)


In [ ]:
# Random test image
if True:
    fig, ax = plt.subplots(5,5, figsize=(18,18))
    for i in range (5):
        for j in range (5):
            e = df[df.FaceCount==0].sample().iloc[0]
            path = e.Path
            img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
            ax[i][j].imshow(img)

    plt.suptitle("Failed")

In [ ]:
dfX = df[df.FaceCount==0]
yaw = dfX.GazeAngle.apply(lambda x: x[0]).to_list()
pitch = dfX.GazeAngle.apply(lambda x: x[1]).to_list()
plt.hist(yaw, alpha = .5, label="Yaw")
plt.hist(pitch, alpha = .5, label="Pitch")
plt.legend()
plt.show()
plt.hist2d(yaw, pitch, bins=[np.arange(-180,180,2),np.arange(-90,90,2)], cmap="hot")
plt.colorbar()
plt.show()

In [ ]:
e = df[df.FaceCount==2].sample().iloc[0]
f = e.Face
path = e.Path

img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

fig, ax = plt.subplots(1, 1)
plt.imshow(img)
ax.add_patch(patches.Rectangle((f[0], f[1]), f[2] - f[0], f[3] - f[1], linewidth=2, edgecolor='r', facecolor='none'))
w =  f[2] - f[0]
h = f[3] - f[1]
ax.add_patch(patches.Rectangle((f[0]-w*.4, f[1]-h*.4),w*1.8, h*1.8, linewidth=2, edgecolor='w', facecolor='none'))
            

In [ ]:

def getFaceCrop(path, f, s):
    f = np.array(f).reshape((2,2))
    s = np.array(s[:2])
    SIZE = 500
    f = np.ravel(f/s)
    f = f* SIZE
    # Enlarge face bounderyby 40%, needs to be applyed to evey new dataset / image
    w =  (f[2] - f[0])*.4
    h = (f[3] - f[1])*.4
    f = np.round([f[0]-w, f[1]-h, f[2]+w, f[3]+h]).astype(int)

    left, top, right, bottom = f
    im = Image.open(path) 
    im = im.resize((SIZE,SIZE))
    im = im.crop((left, top, right, bottom)) 
    return im.resize((224,224))
    
    #img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    #img2 = cv2.resize(img, (SIZE,SIZE))
    #img2 = img2[f[0]:f[2], f[1]:f[3],:] # This method does not pad incase of minus values or larger faces than the image
    #return img2
    #return cv2.resize(img2, (224,224))

In [ ]:
PATH_IMAGES_NEW = "./data/imgs_v2/"

In [ ]:
def generateImageCrops(e):
    if e.Face is None:
        return None
    else:
        path = "%srec_%03d/head/%06d/%06d.jpg" % (PATH_IMAGES_NEW, e.Recording, e.PersonIdentity, e.Frame)
        im = getFaceCrop(e.Path, e.Face, e.Size)
        pathf_folder = "%srec_%03d/head/%06d/" % (PATH_IMAGES_NEW, e.Recording, e.PersonIdentity)
        path = "%s%06d.jpg" % (pathf_folder, e.Frame)
        os.makedirs(pathf_folder, exist_ok=True)
        im.save(path)
        return path

In [ ]:
dfX = df[df.FaceCount>0].sample()
print(path)
img = cv2.cvtColor(cv2.imread(dfX.iloc[0].Path), cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()
x = dfX.apply(lambda e: generateImageCrops(e), axis=1)
path = x.iloc[0]
print(path)
img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
%%time
df["PathNew"] = df.progress_apply(lambda e: generateImageCrops(e), axis=1)

In [ ]:
df.to_pickle("./data/metadata_v2.pkl")

In [ ]:
df = pd.read_pickle("./data/metadata_v2.pkl")

In [ ]:
df = df[df.FaceCount > 0]
df = df.sort_values(["Recording", "PersonIdentity", "Frame"])
df.index = list(range(len(df)))


For the training we do not want to have to many or may be no gaps in the 7 frames.
Checking for the amount of gaps in the follwoing.

In [ ]:
df["RecordingDiff"] = df.Recording.diff()
df["PersonIdentityDiff"] = df.PersonIdentity.diff()
df["FrameDiff"] = df.Frame.diff()

In [ ]:
len(df)

In [ ]:
dfX =df[(df.RecordingDiff == 0) & (df.PersonIdentityDiff == 0) & (df.FrameDiff != 1)]
print("Gaps within one tracked face: %i (%.2f%%)" % (len(dfX), len(dfX)/len(df)*100))
dfX.FrameDiff.hist()
plt.yscale('log')
plt.xscale('log')

In [ ]:
%%time

#
# This works but is super slow ETA ~2h
# To much slicing is happening here
#
#def countMissingFrames(e):
#    dfX = df[(df.Recording == e.Recording) & (df.PersonIdentity == e.PersonIdentity) & (df.Frame >= e.Frame-3) & (df.Frame <= e.Frame+3)]
#    return 7 - len(dfX)
#df["MissingCount"] = df.progress_apply(lambda e: countMissingFrames(e), axis = 1)

def countMissingFrames(df, f):
    dfX = df[(df.Frame >= f-3) & (df.Frame <= f+3)]
    return 7 - len(dfX)
lst = []
for i, r in enumerate(sorted(df.Recording.unique())):
    if (i % 5 == 0):
        print("%s: itr %i, Running recoding %i" % (datetime.now(), i, r))
    df1 = df[(df.Recording == r)]
    for p in sorted(df1.PersonIdentity.unique()):
        df2 = df1[(df1.PersonIdentity == p)]
        ret = df2.Frame.apply(lambda x: countMissingFrames(df2, x))
        lst.append(ret)
        
df["MissingCount"] = pd.concat(lst)

In [ ]:
for i in range(0, 7):
    length = len(df[df.MissingCount == i])
    print("%i missing images for %i images %.2f%%" % (i, length, length/ len(df)* 100))
df.MissingCount.hist(bins=6)

In [ ]:
df.groupby(["Split", "MissingCount"]).count()

In [ ]:
df.to_pickle("./data/metadata_v3.pkl")

There is very little meaing to the training process when there are lots of images missing. Thus, we will remove them from the training list. However, not from the images itself as this will create even more missing images. 

We will showcase in the following what this means with respect to training the data dencity.

In [ ]:
dfFilter = df[df.MissingCount < 3]
print (" This removed %.2f%% of the samples" % ((1-(len(dfFilter)/ len(df)))* 100))

In [ ]:
print("Data split by sample in %")
(dfFilter.groupby("Split").count().Frame / len(dfFilter)*100).round(2)

In [ ]:
print("Data split by number of people indentified - this is not the particiapnts count!")
dfFilter.groupby("Split").PersonIdentity.unique().apply(lambda x: len(x))

In [ ]:
dfX = dfFilter[dfFilter.Split == "train"]
yaw = dfX.GazeAngle.apply(lambda x: x[0]).to_list()
pitch = dfX.GazeAngle.apply(lambda x: x[1]).to_list()
plt.hist(yaw, alpha = .5, label="Yaw")
plt.hist(pitch, alpha = .5, label="Pitch")
plt.legend()
plt.show()
plt.hist2d(yaw, pitch, bins=[np.arange(-180,180,2),np.arange(-90,90,2)], cmap="hot")
plt.colorbar()
plt.show()

In [ ]:
dfX = dfFilter[dfFilter.Split == "val"]
yaw = dfX.GazeAngle.apply(lambda x: x[0]).to_list()
pitch = dfX.GazeAngle.apply(lambda x: x[1]).to_list()
plt.hist(yaw, alpha = .5, label="Yaw")
plt.hist(pitch, alpha = .5, label="Pitch")
plt.legend()
plt.show()
plt.hist2d(yaw, pitch, bins=[np.arange(-180,180,2),np.arange(-90,90,2)], cmap="hot")
plt.colorbar()
plt.show()

In [ ]:
dfX = dfFilter[dfFilter.Split == "test"]
yaw = dfX.GazeAngle.apply(lambda x: x[0]).to_list()
pitch = dfX.GazeAngle.apply(lambda x: x[1]).to_list()
plt.hist(yaw, alpha = .5, label="Yaw")
plt.hist(pitch, alpha = .5, label="Pitch")
plt.legend()
plt.show()
plt.hist2d(yaw, pitch, bins=[np.arange(-180,180,2),np.arange(-90,90,2)], cmap="hot")
plt.colorbar()
plt.show()

In [ ]:
e = df.sample().iloc[0]
print(e.Path)

In [ ]:
 from scipy import stats

In [ ]:
des = stats.describe(img.flatten())
des

In [ ]:
def imageInfo(path):
    img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    img.flatten()
    img = img.flatten()
    return {"Mean":np.mean(img), "SD":np.std(img), "Min":np.min(img), "Max":np.max(img),  "Skewness":scipy.stats.skew(img)}
                                                                                                                  
                                                                                                                  

In [ ]:
info = df.Path.progress_apply(lambda x: imageInfo(x))

In [ ]:
dfInfo = pd.DataFrame(info.tolist())
dfInfo["Frame"] = df.Frame
dfInfo.head()

In [ ]:
dfInfo.Mean.hist(label="Mean", bins=25, alpha=.5)
dfInfo.Max.hist(label="Max", bins=25, alpha=.5)
plt.xlim(0,255)
plt.legend()

In [ ]:
img = cv2.cvtColor(cv2.imread(e.PathNew), cv2.COLOR_BGR2RGB)
equ = cv2.equalizeHist( cv2.cvtColor(img, cv2.COLOR_RGB2GRAY))

fig, ax = plt.subplots(1,4, figsize=(18,4))
ax[0].imshow(img)
ax[1].imshow(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY), cmap="gray")
ax[2].imshow(equ, cmap="gray")

hist,bins = np.histogram(img.flatten(),256,[0,256])

cdf = hist.cumsum()
cdf_normalized = cdf * hist.max()/ cdf.max()

ax[3].plot(cdf_normalized, color = 'b')
ax[3].hist(img.flatten(),256,[0,256], color = 'r')